In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import pandas as pd
import os 
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from langchain.document_loaders import DataFrameLoader

/tmp/ipykernel_2661/2555632222.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# на тестовом

In [10]:
# documents = [
#     {"id": "1", "question": "Как восстановить пароль?", "answer": "Если вы забыли пароль, воспользуйтесь службой восстановления. В отправленном вам письме будет ссылка на смену пароля.", 'url': ""},
#     {"id": "2", "question": "Как сменить логин?", "answer": "Логин изменить невозможно. Логин есть у тех пользователей, кто зарегистрировался на сайте до мая 2015 года. Участники, которые зарегистрировались позже, при авторизации вместо логина указывают свою электронную почту.", "url": ""}
# ]

# # создаем из наших документов датафрейм
# df = pd.DataFrame(documents)
# df.head()

In [2]:
# грузим фрейм в лоадер, выделив колонку для векторизации (здесь может быть место для дискуссий)
df = pd.read_csv("./SOURCE_DOCUMENTS/questions_stolot.csv", dtype='object')
loader = DataFrameLoader(df, page_content_column='question')
documents = loader.load()

In [3]:
rec_text_splitter = RecursiveCharacterTextSplitter(chunk_size=700,
                                               chunk_overlap=700,
                                               length_function=len
                                               )
texts = rec_text_splitter.split_documents(documents)

# <span style="color:orange">**=== chromadb ===**</span>
В этой вариации я использую векторный поиск не по всему тексту, а по документам, в которых поиск выполняется только по полю "question"

In [4]:
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")


chroma_client = chromadb.HttpClient(settings=Settings(
    allow_reset=True,
    chroma_api_impl='chromadb.api.fastapi.FastAPI',
    chroma_server_host='localhost',
    chroma_server_http_port='8000')
)
chroma_client.delete_collection('book')
collection = chroma_client.create_collection(name='book',
                                             metadata={"hnsw:space": "cosine"},
                                             embedding_function=embedding_function
                                             )

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
counter = 0
for doc in texts:
    counter += 1
    print(f'Done: {round(counter * 100 / len(texts), 2)}%')
    collection.add(
        documents=doc.page_content,
        metadatas=doc.metadata,
        ids=doc.metadata['id']
    )

Done: 0.74%
Done: 1.48%
Done: 2.22%
Done: 2.96%
Done: 3.7%
Done: 4.44%
Done: 5.19%
Done: 5.93%
Done: 6.67%
Done: 7.41%
Done: 8.15%
Done: 8.89%
Done: 9.63%
Done: 10.37%
Done: 11.11%
Done: 11.85%
Done: 12.59%
Done: 13.33%
Done: 14.07%
Done: 14.81%
Done: 15.56%
Done: 16.3%
Done: 17.04%
Done: 17.78%
Done: 18.52%
Done: 19.26%
Done: 20.0%
Done: 20.74%
Done: 21.48%
Done: 22.22%
Done: 22.96%
Done: 23.7%
Done: 24.44%
Done: 25.19%
Done: 25.93%
Done: 26.67%
Done: 27.41%
Done: 28.15%
Done: 28.89%
Done: 29.63%
Done: 30.37%
Done: 31.11%
Done: 31.85%
Done: 32.59%
Done: 33.33%
Done: 34.07%
Done: 34.81%
Done: 35.56%
Done: 36.3%
Done: 37.04%
Done: 37.78%
Done: 38.52%
Done: 39.26%
Done: 40.0%
Done: 40.74%
Done: 41.48%
Done: 42.22%
Done: 42.96%
Done: 43.7%
Done: 44.44%
Done: 45.19%
Done: 45.93%
Done: 46.67%
Done: 47.41%
Done: 48.15%
Done: 48.89%
Done: 49.63%
Done: 50.37%
Done: 51.11%
Done: 51.85%
Done: 52.59%
Done: 53.33%
Done: 54.07%
Done: 54.81%
Done: 55.56%
Done: 56.3%
Done: 57.04%
Done: 57.78%
Done: 5

In [6]:
# question = "Если я иностранец, то могу ли участвовать в лотереях?"
# question = "хочу сменить пароль пароль"
question = 'Почему я не могу купить билет?'

response = collection.query(
    # query_embeddings=embedding_function(question),
    query_texts=question,
    n_results=3,
    # include=["documents"],
    # where={"metadata_field":"answer"}, # где искать
    # where_document={"$contains":"$search_string"}
)

# убираем одинаковые ответы
response_list = []
for repl in response['metadatas'][0]:
    response_list.append(repl['answer'])

response_list = set(response_list)

# собираем итоговый ответ
vector_db_response = ""
for repl in response_list:
    vector_db_response += repl
    # if repl['url'] is not False:
    #     full_response += f" Ссылка на подробную информацию: {repl['url']}\n"
    # else:
    #     full_response += "\n"

print(f'============ QUESTION: ============\n{question}')
print(f'============ VECTOR DB RESPONSE: ============\n{vector_db_response}')

============ QUESTION: ============
Почему я не могу купить билет?
============ VECTOR DB RESPONSE: ============
Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.


In [7]:
response

{'ids': [['36', '37', '38']],
 'distances': [[0.07231467962265015,
   0.16297310590744019,
   0.17978370189666748]],
 'embeddings': None,
 'metadatas': [[{'answer': 'Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.',
    'id': '36',
    'url': 'https://www.stoloto.ru/lottery/onlinesale?int=faq'},
   {'answer': 'Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.',
    'id': '37',
    'url': 'https://www.stoloto.ru/lottery/onlinesale?int=faq'},
   {'answer': 'Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.',
    'id': '38',
    'url': 'https://www.stoloto.ru/lottery/onlinesale?int=faq'}]],
 'documents': [['Не могу купить би

# <span style="color:orange">**=== LLM ===**</span>

In [8]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [9]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# SYSTEM_PROMPT = f"""You are a friendly chatbot assistant that responds in a conversational manner to users' questions. \
# Response in 1-2 complete sentences, unless specifically asked by the user to elaborate on something. \
# Use Context only to inform your answers. \
# If you can't find relevant information in the context say that you don't know the answer."""
# SYSTEM_PROMPT = "Ты - злой пират. Отвечай как злобный пират!"
# SYSTEM_PROMPT = "Ты - высокомерный ученый. Отвечай только на вопросы, связанные с наукой и ни на что больше"
# SYSTEM_PROMPT = "Ты - трехлетний ребенок. Отвечай как трехлетний ребенок и только на те вопросы, которые способен знать ребенок в возрасте трех лет."
SYSTEM_PROMPT = f"Ты - доброжелательный русскоязычный ассистент Изя. Ты отвечаешь на вопрос, используя только следующую данную тебе информацию.\nИнформация: {vector_db_response}"


# QUESTION = 'Привет, как тебя зовут?'
# QUESTION = 'Почему?'
# QUESTION = 'Что такое нейтрон?'
# QUESTION = 'Напиши короткий рассказ, в котором должны быть слова "Таня", "Мяч"'
# QUESTION = 'what gave monopoly to british ships?'
# QUESTION = 'Tell me a story about sunshine'
QUESTION = question


chat = [
   {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

# SYSTEM_PROMPT = f"Ты - русскоязычный ассистент Сайга. Тебе показывается важная информация и вопрос человека, на который ты должен дать ответ, используя информацию. Если в предоставленной информации нет ответа, то скажи что ты не можешь ответить на вопрос."
# chat = [
#    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
#     {"role": "user", "content": f"Информация: {vector_db_response}\nВопрос:\n{QUESTION}"},
#     {"role": "assistant", "content":""}
# ]

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right' # ???

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'

input_message = input_message[:-5].strip() + "\n"
print(input_message)

<s>system
Ты - доброжелательный русскоязычный ассистент Изя. Ты отвечаешь на вопрос, используя только следующую данную тебе информацию.
Информация: Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.</s>
<s>user
Почему я не могу купить билет?</s>
<s>assistant



In [11]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    # load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda"
)

model.eval()

model = model.merge_and_unload() #  ОБЯЗАТЕЛЬНО!!!

from transformers import GenerationConfig

generation_config = GenerationConfig(
  bos_token_id = 1,
  do_sample = True,
  eos_token_id = 2,
  max_length = 1024,
  repetition_penalty=1.1,
  # length_penalty=0.1,
  no_repeat_ngram_size=15,
  pad_token_id = 2,
  temperature = 0.1,
  # top_p = 0.9,
  # top_k= 40,
  # low_memory=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [01:42<00:00, 51.37s/it]


In [12]:
input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]

output_message = tokenizer.decode(output_data, skip_special_tokens=True)
print(output_message)

Возможно, вы исчерпали ежемесячные лимиты вашего кошелька. Для решения этой проблемы вам нужно пройти идентификацию на сайте Столoto или ожидать первого числа нового месяца для обновления лимитов.


# множество вопросов

In [13]:
questions_list = ['Где надо вводить код выигрыша',
 'Здравствуйте,  почему не выводится сумма выиграша с кошелька?',
 'Добрый день , какая то проблема с выводом средств из кошелька , пишет что перечислено , но не перечисляет  и деньги остаются в кошелке',
 'Здравствуйте почему не могу купить билет оплатив его с кошелька столото?',
 'Позови оператора',
 'Добрый вечер! Не подскажите где выигрыш , 170 руб? Переводил его в кошелек и куда то пропали деньги',
 'Добрый день, У меня в кошельке 313 р. Почему я не могу купить билет?',
 'Добрый день!, Меня интересует сколько денег я потратила на билеты всего за все время, Я третий раз пишу, почему мне не отвечают до сих пор, ??????',
 'Здравствуйте. Почему в разделе победители у всех аватарки одинаковые, фон однотонный цвет, только сам цвет меняется?'
 ]

In [14]:
for question in questions_list:
    response = collection.query(
    # query_embeddings=embedding_function(question),
    query_texts=[question],
    n_results=2,
    # include=["documents"],
    # where={"metadata_field":"is_equal_to_this"}, # где искать
    # where_document={"$contains":"$search_string"}
    )

    vector_db_response = " ".join(response["documents"][0])

    SYSTEM_PROMPT = f"Ты - русскоязычный ассистент Изя. Ты отвечаешь на вопрос пользователя, используя только следующую данную тебе информацию.\nИнформация: {vector_db_response}"
    QUESTION = question

    chat = [
        {"role": "system", "content": f"{SYSTEM_PROMPT}"},
        {"role": "user", "content": f"{QUESTION}"},
        {"role": "assistant", "content": ""}
    ]
    
    input_message = ""
    for i in chat:
        input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'

    input_message = input_message[:-5].strip() + "\n"

    input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
    input_data = {k: v.to("cuda") for k, v in input_data.items()}

    output_data = model.generate(
        **input_data,
        generation_config = generation_config
    )[0]

    output_data = output_data[len(input_data["input_ids"][0]):]
    output_message = tokenizer.decode(output_data, skip_special_tokens=True)

    print(f"ВОПРОС: {question}")
    print(f"==========================")
    print(f"Ответ ассистента: {output_message}")
    print(f"==========================")
    print("\n")

ВОПРОС: Где надо вводить код выигрыша
Ответ ассистента: Код выигрыша обычно вводится в специальной форме или на сайте организации, которая проводит лотерею. Этот код позволяет определить, какой приз выиграл игрок. В зависимости от страны и типа лотереи, процесс может быть немного разным.

Важно обратить внимание, что для получения денежного выплаты вам нужно будет предоставить дополнительные документы, такие как паспорт, справку о месте жительства и т.д.

Если вы хотите найти номер кошелька в своем профиле, то это можно сделать в настройках профиля на сайте или в приложении, где вы регистрировались для участия в лотерее. Номер кошелька обычно содержит информацию о банке, который вы использовали для оплаты участия в лотерее.


ВОПРОС: Здравствуйте,  почему не выводится сумма выиграша с кошелька?
Ответ ассистента: Ваш вопрос не совсем ясен, но возможно вы имеете в виду, что вы хотите узнать о сумме выплаты из вашего кошелька. Если это так, то вам нужно просто перейти в свой личный кабине

# langchain

In [1]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import os 
import chromadb
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = model.merge_and_unload() #  ОБЯЗАТЕЛЬНО!!!
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.43s/it]
/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:75: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnin

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
   

In [33]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
  bos_token_id = 1,
  do_sample = True,
  eos_token_id = 2,
  max_length = 2048,
  repetition_penalty=1.1,
  no_repeat_ngram_size=15,
  pad_token_id = 0,
  temperature = 0.3,
  top_p = 0.9
)

In [34]:
from langchain import PromptTemplate

template = "Ты - русскоязычный ассистент Изя. Ты отвечаешь на вопрос пользователя, используя только данную тебе информацию. Не пытайся придумать ответ.\nИнформация: {context}\nВопрос:{question}\nОтвет:"
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], template='Ты - русскоязычный ассистент Изя. Ты отвечаешь на вопрос пользователя, используя только данную тебе информацию. Не пытайся придумать ответ.\nИнформация: {context}\nВопрос:{question}\nОтвет:')

In [38]:
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

chroma_client = chromadb.HttpClient(settings=Settings(
    allow_reset=True,
    chroma_api_impl='chromadb.api.fastapi.FastAPI',
    chroma_server_host='localhost',
    chroma_server_http_port='8000')
)
collection = chroma_client.get_collection(name="book", embedding_function=embedding_function)

db = Chroma(client=chroma_client,
            collection_name='book',
            embedding_function=embeddings)

retriever = db.as_retriever(
    search_kwargs={"k":2}
)

question = "как тебя зовут"
response = db.similarity_search_with_score(question)
response

[(Document(page_content='Как избавиться от сыгранных билетов в личном кабинете?', metadata={'answer': 'Самостоятельное удаление билетов невозможно — они автоматически перемещаются в архив по истечении 180 дней с момента проведения тиража/розыгрыша.', 'id': '13', 'url': 'False'}),
  0.3138025999069214),
 (Document(page_content='Как купить билет', metadata={'answer': 'Возможно, Вы исчерпали ежемесячные лимиты по Кошельку. Вам необходимо пройти идентификацию на сайте Столото или ожидать первого числа нового месяца, когда лимиты обновятся.', 'id': '38', 'url': 'https://www.stoloto.ru/lottery/onlinesale?int=faq'}),
  0.3264155387878418),
 (Document(page_content='Как изменить пароль в личном кабинете?', metadata={'answer': 'Изменить имя и пароль вы можете в своем личном кабинете (раздел «Профиль»).Если вы забыли пароль, воспользуйтесь службой восстановления. В отправленном вам письме будет ссылка на смену пароля.Логин изменить невозможно. Логин есть у тех пользователей, кто зарегистрировался

In [39]:
pipe = pipeline(
        'text-generation',
        model = model,
        tokenizer = tokenizer,
        generation_config=generation_config
    )

llm = HuggingFacePipeline(pipeline=pipe)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [40]:
print(generation_config)
generated_text = qa.invoke({"query": question})

print(f"============QUESTION:============\n{generated_text['query']}\n")
print(f"============RESPONSE:============\n{generated_text['result']}")

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 2048,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.3,
  "top_p": 0.9
}

============QUESTION:============
как тебя зовут

============RESPONSE:============
 Я - русскоязычный ассистент Изя.


In [36]:
print(generation_config)

for question in questions_list:
    generated_text = qa.invoke({"query": question})

    print(f"ВОПРОС: {generated_text['query']}")
    print(f"==========================")
    print(f"Ответ ассистента: {generated_text['result']}")
    print(f"==========================")
    print("\n")

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 2048,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.3,
  "top_p": 0.9
}

ВОПРОС: Где надо вводить код выигрыша
Ответ ассистента:  В зависимости от конкретного лотерейного оператора, место ввода кода выигрыша может быть разным. Возможно, это будет в секции "Мои выигрыши" или "Проверка результатов". Рекомендуется обратиться к поддержке оператора для получения точной информации.


ВОПРОС: Здравствуйте,  почему не выводится сумма выиграша с кошелька?
Ответ ассистента:  Здравствуйте! Сумма выплаты может быть ограничена для предотвращения мошенничества и совершения незаконных действий. Пожалуйста, обратитесь к поддержке сайта для дополнительной информации.


ВОПРОС: Добрый день , какая то проблема с выводом средств из кошелька , пишет что перечислено , но не перечисляет  и деньги остаются в кошелке
Ответ ассистента:  Пожалуйста, обратитесь в п

### <span style="color:orange">**=== vol.5 with chat history ===**</span>

In [1]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
import os 
import chromadb
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
config = PeftConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda"
)
model = model.merge_and_unload() #  ОБЯЗАТЕЛЬНО!!!
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.80s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [3]:
generation_config = GenerationConfig(
  bos_token_id = 1,
  do_sample = True,
  eos_token_id = 2,
  max_length = 1024,
  repetition_penalty=1.1,
  no_repeat_ngram_size=15,
  pad_token_id = 0,
  temperature = 0.2,
  top_p = 0.9
)

In [4]:
pipe = pipeline(
        'text-generation',
        model = model,
        tokenizer = tokenizer,
        generation_config=generation_config
    )

llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

chroma_client = chromadb.HttpClient(settings=Settings(
    allow_reset=True,
    chroma_api_impl='chromadb.api.fastapi.FastAPI',
    chroma_server_host='localhost',
    chroma_server_http_port='8000')
)

db = Chroma(client=chroma_client,
            collection_name='book',
            embedding_function=embeddings)

retriever = db.as_retriever(
    search_kwargs={"k":3}
)

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Ответь на следующий вопрос, используя только эту информацию:\n{context}. НЕ ГЕНЕРИРУЙ САМОСТОЯТЕЛЬНО ВОПРОСЫ ЗА Human!"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [20]:
retriever_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "На основе диалога выше, сгенерируй поисковый запрос, чтобы найти релевантную информацию")
])

history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=retriever_prompt
)

retrieval_chain = create_retrieval_chain(
    history_aware_retriever,
    chain
)

In [25]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content='Привет, я обращаюсь к тебе за помощью только по вопросам лотерей Столото. Меня интересует только эта информация.'),
    AIMessage(content='Привет, меня зовут Степан, меня создали гениальные ML-разработчики из компании "Синхро". Я готов помочь тебе с твоими вопросами!')
]

input_message = 'Ты можешь мне помочь с моими вопросами?'

response = retrieval_chain.invoke({
        "chat_history": chat_history,
        "input": input_message
    })['answer']
chat_history.append(HumanMessage(content=input_message))
chat_history.append(AIMessage(content=response))

response

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'\nAI: Да, конечно, я готов помочь тебе с любыми вопросами, связанными с лотереями Столото.'

In [26]:
input_message = 'Как меня зовут?'

response = retrieval_chain.invoke({
        "chat_history": chat_history,
        "input": input_message
    })['answer']
chat_history.append(HumanMessage(content=input_message))
chat_history.append(AIMessage(content=response))

response

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'\nAI: Меня зовут Степан, я создан специально для того, чтобы помогать людям с вопросами, связанными с лотереями.'

In [27]:
input_message = 'Что я спрашивал тебя последнее?'

response = retrieval_chain.invoke({
        "chat_history": chat_history,
        "input": input_message
    })['answer']
chat_history.append(HumanMessage(content=input_message))
chat_history.append(AIMessage(content=response))

response

/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/ml_projects/chatbot/9_2_rag/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'\nAI: \nAI: Вы спрашивали меня о том, как посмотреть номер кошелька "Столото" в вашем личном кабинете.'

In [28]:
chat_history

[HumanMessage(content='Привет, я обращаюсь к тебе за помощью только по вопросам лотерей Столото. Меня интересует только эта информация.'),
 AIMessage(content='Привет, меня зовут Степан, меня создали гениальные ML-разработчики из компании "Синхро". Я готов помочь тебе с твоими вопросами!'),
 HumanMessage(content='Ты можешь мне помочь с моими вопросами?'),
 AIMessage(content='\nAI: Да, конечно, я готов помочь тебе с любыми вопросами, связанными с лотереями Столото.'),
 HumanMessage(content='Как меня зовут?'),
 AIMessage(content='\nAI: Меня зовут Степан, я создан специально для того, чтобы помогать людям с вопросами, связанными с лотереями.'),
 HumanMessage(content='Что я спрашивал тебя последнее?'),
 AIMessage(content='\nAI: \nAI: Вы спрашивали меня о том, как посмотреть номер кошелька "Столото" в вашем личном кабинете.')]